In [ ]:
"""
Virtual Makeup / Face Beautifier App (ROI-Based, No MediaPipe)
--------------------------------------------------------------
Features:
- Detect face using Haar Cascade
- Apply simple ROI-based lipstick, blush, and eye shadow
- Adjustable intensity using trackbars
- Toggle effects with keys:
    'l' - toggle lipstick
    'b' - toggle blush
    'e' - toggle eyeshadow
    's' - save image
    'q' - quit

Dependencies:
    pip install opencv-python numpy
"""

import cv2
import numpy as np
import time

# Load face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
if face_cascade.empty():
    print("❌ Error: Haar Cascade XML not loaded! Please check your OpenCV installation.")
    print("Expected file path:", cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    exit()
else:
    print("✅ Haar Cascade loaded successfully.")


# Constants
WINDOW_NAME = "Virtual Makeup App (No MediaPipe)"
SAVE_PREFIX = "virtual_makeup"
LIP_COLOR = (0, 0, 255)       # Red
BLUSH_COLOR = (147, 20, 255)  # Pinkish
EYE_COLOR = (128, 0, 128)     # Purple


def nothing(x):
    pass


def apply_makeup(frame, mask, color, intensity):
    """Apply color on a masked region with adjustable transparency."""
    colored = np.zeros_like(frame)
    colored[:] = color
    mask = cv2.GaussianBlur(mask, (25, 25), 0)
    alpha = (mask / 255.0) * (intensity / 100.0)
    blended = (frame * (1 - alpha[..., None]) + colored * alpha[..., None]).astype(np.uint8)
    return blended


# Create trackbars
cv2.namedWindow(WINDOW_NAME)
cv2.createTrackbar('Lip Intensity', WINDOW_NAME, 70, 100, nothing)
cv2.createTrackbar('Blush Intensity', WINDOW_NAME, 50, 100, nothing)
cv2.createTrackbar('Eye Intensity', WINDOW_NAME, 60, 100, nothing)
cv2.createTrackbar('Smoothness', WINDOW_NAME, 10, 30, nothing)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Camera not detected!")
    exit()

show_lip = True
show_blush = True
show_eye = True
save_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    lip_int = cv2.getTrackbarPos('Lip Intensity', WINDOW_NAME)
    blush_int = cv2.getTrackbarPos('Blush Intensity', WINDOW_NAME)
    eye_int = cv2.getTrackbarPos('Eye Intensity', WINDOW_NAME)
    smooth_val = cv2.getTrackbarPos('Smoothness', WINDOW_NAME)

    for (x, y, w, h) in faces:
        # Draw ROI regions
        face_roi = frame[y:y+h, x:x+w]

        # Define approximate facial regions relative to face box
        lip_y1 = int(y + 0.7*h)
        lip_y2 = int(y + 0.85*h)
        lip_x1 = int(x + 0.25*w)
        lip_x2 = int(x + 0.75*w)

        cheek_y1 = int(y + 0.45*h)
        cheek_y2 = int(y + 0.6*h)
        left_cheek_x1 = int(x + 0.15*w)
        left_cheek_x2 = int(x + 0.35*w)
        right_cheek_x1 = int(x + 0.65*w)
        right_cheek_x2 = int(x + 0.85*w)

        eye_y1 = int(y + 0.25*h)
        eye_y2 = int(y + 0.4*h)
        left_eye_x1 = int(x + 0.2*w)
        left_eye_x2 = int(x + 0.4*w)
        right_eye_x1 = int(x + 0.6*w)
        right_eye_x2 = int(x + 0.8*w)

        mask = np.zeros(frame.shape[:2], np.uint8)

        # Apply makeup regions
        if show_lip:
            cv2.rectangle(mask, (lip_x1, lip_y1), (lip_x2, lip_y2), 255, -1)
            frame = apply_makeup(frame, mask, LIP_COLOR, lip_int)

        if show_blush:
            cv2.circle(mask, (left_cheek_x1 + 20, cheek_y1 + 20), 40, 255, -1)
            cv2.circle(mask, (right_cheek_x1 + 60, cheek_y1 + 20), 40, 255, -1)
            frame = apply_makeup(frame, mask, BLUSH_COLOR, blush_int)

        if show_eye:
            cv2.rectangle(mask, (left_eye_x1, eye_y1), (left_eye_x2, eye_y2), 255, -1)
            cv2.rectangle(mask, (right_eye_x1, eye_y1), (right_eye_x2, eye_y2), 255, -1)
            frame = apply_makeup(frame, mask, EYE_COLOR, eye_int)

        # Beautify (smooth skin using bilateral filter)
        smooth_frame = cv2.bilateralFilter(frame, smooth_val*2+1, 75, 75)
        frame = cv2.addWeighted(frame, 0.6, smooth_frame, 0.4, 0)

    cv2.putText(frame, "Keys: l-Lips | b-Blush | e-Eyes | s-Save | q-Quit",
                (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow(WINDOW_NAME, frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key == ord('l'):
        show_lip = not show_lip
    elif key == ord('b'):
        show_blush = not show_blush
    elif key == ord('e'):
        show_eye = not show_eye
    elif key == ord('s'):
        filename = f"{SAVE_PREFIX}_{int(time.time())}_{save_count}.png"
        cv2.imwrite(filename, frame)
        print(f"Saved {filename}")
        save_count += 1

cap.release()
cv2.destroyAllWindows()


✅ Haar Cascade loaded successfully.
